Member 1 : KEUNG Yat LONG
SID : 5714 6792

Member 2 : Wong Pedro
SID : 56751109

Member 3 : KO Ka Chun
SID : 56744470

#Background
___
- Image generation techniques have existed for decades.
- Recent deep learning advances (GANs, diffusion models) have boosted photorealism in AI-generated content (AIGC).
- While these advancements have entertainment value, they also pose risks of weaponization.
- Detecting AIGC is now a critical issue and a prominent research focus.

#Basic Information
---
- **Main task**: Binary classification (detect whether an image is AI-generated or not).
- **Input**: RGB images.
- **Output**: Binary label indicating if the image is AI-generated.
- **Training set**: 45,000 images.
- **Validation set**: 5,000 images.

#Dataset Exploration
---
- Dataset includes both photographic and AI-generated images.
- Photographic images are sourced from ImageNet with varying sizes.
- AI-generated images are 512 × 512 × 3, created using Stable Diffusion v1.4, trained on the LAION dataset.
- Photographic and AI-generated images have similar semantic content to avoid content bias.
- Only binary labels are available for training and testing.

##Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/CS4487/CS4487 Project/AIGC-Detection-Dataset.zip'

# Create a directory to extract the dataset
output_dir = 'AIGC_Dataset'
os.makedirs(output_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f'Dataset extracted to {output_dir}')

Dataset extracted to AIGC_Dataset


##Data Preprocessing

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Test Code

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from sklearn import metrics
from torch.utils.data import Dataset

In [ ]:
# Define a custom dataset loader for binary classification
class TestDataset(Dataset):
    def __init__(self, data_dir):
        real_dir = os.path.join(data_dir, '0_real')
        fake_dir = os.path.join(data_dir, '1_fake')

        # Load file paths and labels
        self.image_paths = [os.path.join(real_dir, f) for f in os.listdir(real_dir)] + \
                           [os.path.join(fake_dir, f) for f in os.listdir(fake_dir)]
        self.labels = [0] * len(os.listdir(real_dir)) + [1] * len(os.listdir(fake_dir))

        # Image transformations
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize([0.4778, 0.4559, 0.4175], [0.2794, 0.2739, 0.2902])  # Normalize using ImageNet's mean and std
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        return image, label

In [ ]:
# Define the test function
def test(model, test_dataset_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()

    # Load test dataset
    test_dataset = TestDataset(test_dataset_path)
    y_true, y_pred = [], []

    for img, label in test_dataset:
        img = img.unsqueeze(0).to(device)  # Add batch dimension
        output = model(img).item()  # Forward pass

        pred = 1 if output > 0.5 else 0  # Sigmoid threshold
        y_true.append(label)
        y_pred.append(pred)

    # Calculate accuracy
    accuracy = metrics.accuracy_score(y_true, y_pred)
    return accuracy

In [ ]:
# Replace these paths before running the test
test_dataset_path = ''  # Replace with the path to the test dataset folder

In [ ]:
import torch
from torchvision import models
import torch.nn as nn

# Define DenseNet model
def build_densenet_model():
    base_model = models.densenet121(weights=None)  # Initialize without pretrained weights
    num_features = base_model.classifier.in_features
    base_model.classifier = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.8),
        nn.Linear(512, 1),
    )
    return base_model

# Define Swin model
def build_swin_model():
    swin_model = models.swin_t(weights=None)  # Initialize without pretrained weights
    swin_model.head = nn.Linear(swin_model.head.in_features, 1)  # Binary classification
    return swin_model

# Define the Ensemble Model
class EnsembleModel(nn.Module):
    def __init__(self, densenet, swin, classifier):
        super(EnsembleModel, self).__init__()
        self.densenet = densenet
        self.swin = swin
        self.classifier = classifier
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # For DenseNet pooling

    def forward(self, x):
        # DenseNet features
        densenet_features = self.densenet.features(x)
        densenet_pooled = self.global_avg_pool(densenet_features)
        densenet_flattened = torch.flatten(densenet_pooled, start_dim=1)

        # Swin features
        swin_features = self.swin(x).squeeze().unsqueeze(1)

        # Combine features
        combined_features = torch.cat([densenet_flattened, swin_features], dim=1)

        # Classifier
        output = self.classifier(combined_features)
        return output

# Initialize the models and classifier
densenet_model = build_densenet_model()
swin_model = build_swin_model()

ensemble_classifier = nn.Sequential(
    nn.Linear(1025, 512),
    nn.ReLU(),
    nn.Dropout(0.8),
    nn.Linear(512, 1)
)

In [ ]:
# Create Ensemble Model
ensemble_model = EnsembleModel(densenet_model, swin_model, ensemble_classifier).to("cuda")

In [ ]:
# Load the saved state dict
ensemble_model.load_state_dict(torch.load("ensemble_model.pth"))
ensemble_model.eval()

In [ ]:
test_dataset_path = ''

In [ ]:
# Evaluate the model
accuracy = test(ensemble_model, test_dataset_path)
print(f"Test Accuracy: {accuracy}")